In \[17\]:

    import pygame
    import random
    import sys

    pygame.init()

    # Screen setup
    WIDTH, HEIGHT = 600, 800
    screen = pygame.display.set_mode((WIDTH, HEIGHT))
    pygame.display.set_caption("Space Shooter")

    # Colors
    WHITE = (255, 255, 255)
    RED = (255, 0, 0)
    BLUE = (0, 0, 255)
    BLACK = (0, 0, 0)
    ORANGE = (255, 165, 0)

    # Sound effects
    shoot_sound = pygame.mixer.Sound("shoot.wav")
    explosion_sound = pygame.mixer.Sound("explosion.wav")
    hit_sound = pygame.mixer.Sound("hit.wav")
    death_music = pygame.mixer.Sound("death.wav")

    # Clock
    clock = pygame.time.Clock()
    FPS = 60

    # Font
    def draw_text(text, size, color, x, y, center=True):
        font = pygame.font.SysFont(None, size)
        surface = font.render(text, True, color)
        rect = surface.get_rect()
        if center:
            rect.center = (x, y)
        else:
            rect.topleft = (x, y)
        screen.blit(surface, rect)

    def game_over_screen(score):
        death_music.play()
        while True:
            screen.fill(BLACK)
            draw_text("GAME OVER", 72, RED, WIDTH // 2, HEIGHT // 3)
            draw_text(f"Final Score: {score}", 48, WHITE, WIDTH // 2, HEIGHT // 2)
            draw_text("Click to Play Again", 36, WHITE, WIDTH // 2, HEIGHT * 2 // 3)
            pygame.display.flip()

            for event in pygame.event.get():
                if event.type == pygame.QUIT:
                    pygame.quit()
                    sys.exit()
                elif event.type == pygame.MOUSEBUTTONDOWN:
                    death_music.stop()
                    return  # Restart game

    def run_game():
        # Player setup
        player = pygame.Rect(WIDTH // 2 - 25, HEIGHT - 60, 50, 50)
        player_speed = 5
        lives = 3

        # Bullet setup
        bullets = []
        bullet_speed = 7

        # Enemies
        enemies = []
        enemy_speed = 3
        spawn_delay = 40
        spawn_timer = 0

        # Boss
        boss = None
        boss_health = 0
        boss_timer = 0
        boss_cooldown = 60
        boss_bullets = []
        boss_bullet_speed = 4
        boss_active = False

        # Game state
        score = 0
        level = 1

        running = True
        while running:
            clock.tick(FPS)
            screen.fill(BLACK)

            # Increase level every 10 points
            level = score // 10 + 1
            enemy_speed = 2 + level
            spawn_delay = max(20, 40 - level * 2)

            # Event handling
            for event in pygame.event.get():
                if event.type == pygame.QUIT:
                    pygame.quit()
                    sys.exit()

            # Movement
            keys = pygame.key.get_pressed()
            if keys[pygame.K_LEFT] and player.left > 0:
                player.x -= player_speed
            if keys[pygame.K_RIGHT] and player.right < WIDTH:
                player.x += player_speed
            if keys[pygame.K_SPACE]:
                if len(bullets) < 5:
                    bullets.append(pygame.Rect(player.centerx - 2, player.top, 4, 10))
                    shoot_sound.play()

            # Update bullets
            for bullet in bullets[:]:
                bullet.y -= bullet_speed
                if bullet.bottom < 0:
                    bullets.remove(bullet)

            # Spawn enemies
            spawn_timer += 1
            if spawn_timer >= spawn_delay and not boss_active:
                spawn_timer = 0
                rect = pygame.Rect(random.randint(0, WIDTH - 40), 0, 40, 40)
                health = random.choice([1, 2])
                enemies.append({'rect': rect, 'health': health})

            # Update enemies
            for enemy in enemies[:]:
                enemy['rect'].y += enemy_speed
                if enemy['rect'].top > HEIGHT:
                    enemies.remove(enemy)
                elif enemy['rect'].colliderect(player):
                    lives -= 1
                    hit_sound.play()
                    enemies.remove(enemy)
                    if lives <= 0:
                        game_over_screen(score)
                        return

            # Bullet collisions
            for bullet in bullets[:]:
                for enemy in enemies[:]:
                    if bullet.colliderect(enemy['rect']):
                        bullets.remove(bullet)
                        enemy['health'] -= 1
                        hit_sound.play()
                        if enemy['health'] <= 0:
                            enemies.remove(enemy)
                            score += 1
                            explosion_sound.play()
                        break

            # Boss spawn
            if level % 5 == 0 and not boss_active:
                boss = pygame.Rect(WIDTH//2 - 60, 50, 120, 60)
                boss_health = 20
                boss_active = True
                boss_timer = 0
                boss_bullets.clear()

            # Boss logic
            if boss_active:
                boss_timer += 1
                boss.x += random.choice([-2, 0, 2])
                boss.x = max(0, min(WIDTH - boss.width, boss.x))

                if boss_timer % boss_cooldown == 0:
                    bullet = pygame.Rect(boss.centerx - 5, boss.bottom, 10, 20)
                    boss_bullets.append(bullet)

                for b in boss_bullets[:]:
                    b.y += boss_bullet_speed
                    if b.top > HEIGHT:
                        boss_bullets.remove(b)
                    elif b.colliderect(player):
                        lives -= 1
                        hit_sound.play()
                        boss_bullets.remove(b)
                        if lives <= 0:
                            game_over_screen(score)
                            return

                for bullet in bullets[:]:
                    if bullet.colliderect(boss):
                        bullets.remove(bullet)
                        boss_health -= 1
                        hit_sound.play()
                        if boss_health <= 0:
                            boss_active = False
                            boss = None
                            boss_bullets.clear()
                            score += 10
                            explosion_sound.play()
                        break

            # Draw player
            pygame.draw.rect(screen, BLUE, player)

            # Draw bullets
            for bullet in bullets:
                pygame.draw.rect(screen, WHITE, bullet)

            # Draw enemies
            for enemy in enemies:
                color = RED if enemy['health'] == 1 else ORANGE
                pygame.draw.rect(screen, color, enemy['rect'])

            # Draw boss
            if boss_active and boss:
                pygame.draw.rect(screen, (200, 0, 200), boss)
                draw_text(f"Boss HP: {boss_health}", 24, WHITE, boss.centerx, boss.top - 20)

            # Draw boss bullets
            for b in boss_bullets:
                pygame.draw.rect(screen, RED, b)

            # Draw score/lives/level
            draw_text(f"Score: {score}", 30, WHITE, 10, 10, center=False)
            draw_text(f"Lives: {lives}", 30, WHITE, WIDTH - 120, 10, center=False)
            draw_text(f"Level: {level}", 30, WHITE, WIDTH // 2, 10)

            pygame.display.flip()

    # Start game
    while True:
        run_game()

    An exception has occurred, use %tb to see the full traceback.

    SystemExit

In \[ \]: